In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers,losses,layers
import pickle
from glob import glob
from os.path import basename
import networkx as nx

In [2]:
dataset = 'AIDS700nef'

In [3]:
def load_generated_graphs(dataset_name, file_name='generated_graph_500'):

    dir = './dataset/' + dataset_name + '/' + file_name
    g = open(dir, 'rb')
    generated_graphs = pickle.load(g)
    g.close()
    return generated_graphs



training_pairs = load_generated_graphs(dataset, file_name='generated_graph_pairs')




In [5]:
def sorted_nicely(l):
    def tryint(s):
        try:
            return int(s)
        except:
            return s

    import re
    def alphanum_key(s):
        return [tryint(c) for c in re.split('([0-9]+)', s)]

    return sorted(l, key=alphanum_key)

def load_graphs(dataset_name, train_or_test='train'):
    graphs = []
    dir = './dataset/' + dataset_name + '/' + train_or_test
    for file in sorted_nicely(glob(dir + '/*.gexf')):
        gid = int(basename(file).split('.')[0])
        g = nx.read_gexf(file)
        g.graph['gid'] = gid
        graphs.append(g)
        if not nx.is_connected(g):
            raise RuntimeError('{} not connected'.format(gid))
    return graphs

training_graphs = load_graphs(dataset, train_or_test='train')
testing_graphs = load_graphs(dataset, train_or_test='test')


In [4]:
print(training_graphs[1],testing_graphs[1],training_pairs[1])

NameError: name 'training_graphs' is not defined

In [19]:
class TensorNetworkModule(tf.keras.Model):

    def __init__(self, input_dim):

        super().__init__()
        self.input_dim = input_dim
        self.setup_weights()

    def setup_weights(self):
        initializer = tf.keras.initializers.GlorotUniform()

        self.weight_matrix = initializer(shape = (self.input_dim, self.input_dim, 16), dtype = tf.float32)

        self.weight_matrix_block = initializer(shape= (16, 2*self.input_dim), dtype = tf.float32)

        self.bias = initializer(shape = (16, 1), dtype = tf.float32)


    def call(self, embedding_1, embedding_2):

    
        
        scoring = tf.matmul(tf.transpose(embedding_1, perm=[1,0]), self.weight_matrix.reshape(self.input_dim, -1))
        scoring = scoring.reshape(self.input_dim, 16)
        scoring = tf.matmul(tf.transpose(scoring, perm=[1,0]), embedding_2)
        combined_representation = tf.concat((embedding_1,embedding_2), axis=0)
        block_scoring = tf.matmul(self.weight_matrix_block, combined_representation)
        scores = tf.nn.relu(scoring + block_scoring + self.bias)
        scores = tf.transpose(scores, perm=[1,0])
        return scores

In [14]:
class GraphAggregationLayer(tf.keras.Model):

    def __init__(self,in_features=10, out_features=10):
        super(GraphAggregationLayer, self).__init__()

    def call(self, input, adj):
        h_prime = tf.matmul(adj, input)

        return h_prime

In [30]:
class MyModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    
    g = open('./dataset/' + dataset + '/global_labels', 'rb')
    global_labels = pickle.load(g)
    g.close()
    number_of_labels = len(global_labels)
    self.input_dim = 2* number_of_labels
    
    self.gal1 = GraphAggregationLayer()
    self.gal2 = GraphAggregationLayer()
    self.tensor_network_nc = TensorNetworkModule( 2*number_of_labels)
    self.tensor_network_in = TensorNetworkModule(2*number_of_labels)
    self.tensor_network_ie = TensorNetworkModule(2*number_of_labels)
    
    self.input_nc = tf.keras.Input(shape=(16,))
    self.fully_connected_first_nc = tf.keras.layers.Dense(16, activation="relu", kernel_regularizer='l2')
    self.fully_connected_second_nc = tf.keras.layers.Dense(8, activation="relu", kernel_regularizer='l2')
    self.fully_connected_third_nc = tf.keras.layers.Dense(4, activation="relu", kernel_regularizer='l2')
    self.scoring_layer_nc = tf.keras.layers.Dense(1, activation="sigmoid")
    self.scoring_layer_nc = keras.Sequential([
        self.input_nc, 
        self.fully_connected_first_nc,
        tf.keras.layers.Dropout(0.5),
        self.fully_connected_second_nc,
        tf.keras.layers.Dropout(0.5),
        self.fully_connected_third_nc,
        tf.keras.layers.Dropout(0.3),
        self.scoring_layer_nc])
    
    
    
    self.input_in = tf.keras.Input(shape=(16,) )
    self.fully_connected_first_in = tf.keras.layers.Dense(16, activation="relu", kernel_regularizer='l2')
    self.fully_connected_second_in = tf.keras.layers.Dense(8, activation="relu", kernel_regularizer='l2')
    self.fully_connected_third_in = tf.keras.layers.Dense(4, activation="relu", kernel_regularizer='l2')
    self.scoring_layer_in = tf.keras.layers.Dense(1, activation="sigmoid")
    self.scoring_layer_in = keras.Sequential([
        self.input_in, ##feature_count = tensor_neurons = 16
        self.fully_connected_first_in,
        tf.keras.layers.Dropout(0.5),
        self.fully_connected_second_in,
        tf.keras.layers.Dropout(0.5),
        self.fully_connected_third_in,
        tf.keras.layers.Dropout(0.3),
        self.scoring_layer_in])
    
    self.input_ie = tf.keras.Input(shape=(16,))
    self.fully_connected_first_ie = tf.keras.layers.Dense(16, activation="relu", kernel_regularizer='l2')
    self.fully_connected_second_ie = tf.keras.layers.Dense(8, activation="relu", kernel_regularizer='l2')
    self.fully_connected_third_ie = tf.keras.layers.Dense(4, activation="relu", kernel_regularizer='l2')
    self.scoring_layer_ie = tf.keras.layers.Dense(1, activation="sigmoid")
    self.scoring_layer_ie = keras.Sequential([
        self.input_ie, ##feature_count = tensor_neurons = 16
        self.fully_connected_first_ie,
        tf.keras.layers.Dropout(0.5),
        self.fully_connected_second_ie,
        tf.keras.layers.Dropout(0.5),
        self.fully_connected_third_ie,
        tf.keras.layers.Dropout(0.3),
        self.scoring_layer_ie])
    
    
    

    
  def gal_pass(self, edge_index, features):

    hidden1 = self.gal1(features, edge_index)
    hidden2 = self.gal2(hidden1, edge_index)
    
    return hidden1, hidden2

  def call(self, data):
    
    adj_1 = tf.constant(np.array(data["edge_index_1"].todense()),dtype=tf.float32)
    adj_2 = tf.constant(np.array(data["edge_index_2"].todense()),dtype=tf.float32)

    features_1, features_2 = data["features_1"], data["features_2"]
    
    #GAL 
    graph1_hidden1, graph1_hidden2 = self.gal_pass(adj_1, features_1)#
    graph2_hidden1, graph2_hidden2 = self.gal_pass(adj_2, features_2)#
    


    graph1_01concat = tf.concat([features_1, graph1_hidden1], axis=1)
    graph2_01concat = tf.concat([features_2, graph2_hidden1], axis=1)
    graph1_12concat = tf.concat([graph1_hidden1, graph1_hidden2], axis=1)
    graph2_12concat = tf.concat([graph2_hidden1, graph2_hidden2], axis=1)

    graph1_01pooled = tf.expand_dims(tf.reduce_sum(graph1_01concat,axis=0),1)
    graph1_12pooled = tf.expand_dims(tf.reduce_sum(graph1_12concat,axis=0),1)
    graph2_01pooled = tf.expand_dims(tf.reduce_sum(graph2_01concat,axis=0),1)
    graph2_12pooled = tf.expand_dims(tf.reduce_sum(graph2_12concat,axis=0),1)
    
    

    scores_nc = self.tensor_network_nc(graph1_01pooled, graph2_01pooled)
    scores_nc = self.scoring_layer_nc(scores_nc)

    
   

    

    scores_in = self.tensor_network_in(graph1_01pooled, graph2_01pooled)
    scores_in = self.scoring_layer_in(scores_in)

    scores_ie = self.tensor_network_ie(graph1_12pooled, graph2_12pooled)


    
    scores_ie = self.scoring_layer_ie(scores_ie)
    
   #
    #print("score_ie: \n", scores_ie)
    return(tf.concat([scores_nc, scores_in, scores_ie], axis=1))
    
    



In [15]:
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
def transfer_to_torch(data, type_specified=True):
        g = open('./dataset/' + dataset + '/global_labels', 'rb')
        global_labels = pickle.load(g)
        g.close()
        new_data = dict()
        graph1, graph2 = data['graph_pair'][0], data['graph_pair'][1]
        nodes1, nodes2 = list(graph1.nodes()), list(graph2.nodes())

        features_1, features_2 = [], []

        for n in graph1.nodes():
            features_1.append([1.0 if global_labels[graph1.nodes()[n]['type']] == i else 0.0 for i in global_labels.values()])

        for n in graph2.nodes():
            features_2.append([1.0 if global_labels[graph2.nodes()[n]['type']] == i else 0.0 for i in global_labels.values()])

        features_1, features_2 = tf.constant(np.array(features_1),dtype = tf.float32), tf.constant(np.array(features_2),dtype = tf.float32)

        new_data["edge_index_1"], new_data["edge_index_2"] = nx.adjacency_matrix(graph1), nx.adjacency_matrix(graph2)
        new_data["features_1"], new_data["features_2"] = features_1, features_2

        if(type_specified):
            norm_ged = [data['ged'][key] / (0.5 * (graph1.number_of_nodes() + graph2.number_of_nodes())) for key in ['nc', 'in', 'ie']]
            norm_ged = np.array(norm_ged)
            new_data["target"] = tf.cast(tf.convert_to_tensor(np.exp(-norm_ged)).reshape(1, -1), tf.float32)
            
            norm_gt_ged = (data['ged']['nc'] + data['ged']['in'] + data['ged']['ie']) / (0.5 * (graph1.number_of_nodes() + graph2.number_of_nodes()))
            new_data["gt_ged"] = tf.cast(tf.convert_to_tensor(np.exp(-norm_gt_ged).reshape(1, 1)).reshape(1, -1),tf.float32)
        else:
            norm_gt_ged = data['ged'] / (0.5 * (graph1.number_of_nodes() + graph2.number_of_nodes()))
            new_data["gt_ged"] = tf.cast(tf.convert_to_tensor(np.exp(-norm_gt_ged).reshape(1, 1)).reshape(1, -1),tf.float32)

        return new_data



In [31]:

    
batch_size = 128
batches = []
mse = tf.keras.losses.MeanSquaredError()
cossim = tf.keras.losses.CosineSimilarity()
optimizer = keras.optimizers.Adam(learning_rate = 1e-2)
model = MyModel()
for graph in range(0, len(training_pairs), batch_size):
      batches.append(training_pairs[graph:graph+batch_size])
        
        
iteration = 0
for batch in batches:      
    losses = 0  
    with tf.GradientTape(persistent=True) as tape:
        for graph_pair in batch:
            data = transfer_to_torch(graph_pair)
  
            test = model(data, training=True)
           # print(test)
            losses += mse(data["target"], test)
            
    grads = tape.gradient(losses, model.trainable_weights)
    #print(model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    loss = losses.numpy()    
    print ("Iteration: ", iteration, "loss: ", loss/len(batch))  
    iteration+=1

    
test_scores = []
test_gt_ged = load_generated_graphs(dataset, file_name='ged_matrix_test')
for graph_1 in testing_graphs:
    for graph_2 in training_graphs:
        if((graph_1.graph['gid'], graph_2.graph['gid']) in test_gt_ged):
            curr_graph_pair = {'graph_pair': [graph_1, graph_2], 'ged':test_gt_ged[(graph_1.graph['gid'], graph_2.graph['gid'])]}
            data = transfer_to_torch(curr_graph_pair, type_specified=False)
            prediction = model(data,training=False)
            prediction = tf.math.exp(tf.math.reduce_sum(tf.math.log(prediction))).reshape(1, -1)
            current_error = mse(prediction, data["gt_ged"])
            test_scores.append(current_error)


    for graph_2 in testing_graphs:
        if((graph_1.graph['gid'], graph_2.graph['gid']) in test_gt_ged):
            curr_graph_pair = {'graph_pair': [graph_1, graph_2], 'ged':test_gt_ged[(graph_1.graph['gid'], graph_2.graph['gid'])]}
            data = transfer_to_torch(curr_graph_pair, type_specified=False)
            prediction = model(data,training=False)
            prediction = tf.math.exp(tf.math.reduce_sum(tf.math.log(prediction))).reshape(1, -1)
            current_error = mse(prediction, data["gt_ged"])
            test_scores.append(current_error)

model_error = sum(test_scores) / len(test_scores)
print("\nModel test error: " + str(model_error))

C:\Users\huang\AppData\Local\Temp\ipykernel_13156\157768131.py:21: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  new_data["edge_index_1"], new_data["edge_index_2"] = nx.adjacency_matrix(graph1), nx.adjacency_matrix(graph2)


Iteration:  0 loss:  0.12144497036933899
Iteration:  1 loss:  0.10983861982822418
Iteration:  2 loss:  0.09103536605834961
Iteration:  3 loss:  0.09026333689689636
Iteration:  4 loss:  0.07439111173152924
Iteration:  5 loss:  0.07091686874628067
Iteration:  6 loss:  0.06940209865570068
Iteration:  7 loss:  0.062353745102882385
Iteration:  8 loss:  0.04948916286230087
Iteration:  9 loss:  0.05520903691649437
Iteration:  10 loss:  0.05045954883098602
Iteration:  11 loss:  0.042805589735507965
Iteration:  12 loss:  0.046508584171533585
Iteration:  13 loss:  0.041993074119091034
Iteration:  14 loss:  0.036387309432029724
Iteration:  15 loss:  0.03842842951416969
Iteration:  16 loss:  0.03589348867535591
Iteration:  17 loss:  0.03868696838617325
Iteration:  18 loss:  0.035091232508420944
Iteration:  19 loss:  0.03642066568136215
Iteration:  20 loss:  0.029510146006941795
Iteration:  21 loss:  0.03133748099207878
Iteration:  22 loss:  0.027959346771240234
Iteration:  23 loss:  0.027882052585

Iteration:  190 loss:  0.009572543203830719
Iteration:  191 loss:  0.008479790762066841
Iteration:  192 loss:  0.00688283471390605
Iteration:  193 loss:  0.007844479754567146
Iteration:  194 loss:  0.008376275189220905
Iteration:  195 loss:  0.007976631633937359
Iteration:  196 loss:  0.007318259682506323
Iteration:  197 loss:  0.007003404665738344
Iteration:  198 loss:  0.007884702645242214
Iteration:  199 loss:  0.007305547129362822
Iteration:  200 loss:  0.007403187919408083
Iteration:  201 loss:  0.007741170469671488
Iteration:  202 loss:  0.00784634705632925
Iteration:  203 loss:  0.006823928561061621
Iteration:  204 loss:  0.007095615845173597
Iteration:  205 loss:  0.007300660014152527
Iteration:  206 loss:  0.00727043254300952
Iteration:  207 loss:  0.00763570936396718
Iteration:  208 loss:  0.007775216829031706
Iteration:  209 loss:  0.00908636674284935
Iteration:  210 loss:  0.02615322358906269
Iteration:  211 loss:  0.028548603877425194
Iteration:  212 loss:  0.0261434335261

Iteration:  377 loss:  0.007622045930474997
Iteration:  378 loss:  0.008835313841700554
Iteration:  379 loss:  0.00814378447830677
Iteration:  380 loss:  0.00782507099211216
Iteration:  381 loss:  0.008464409969747066
Iteration:  382 loss:  0.008098236285150051
Iteration:  383 loss:  0.007555454503744841
Iteration:  384 loss:  0.007881629280745983
Iteration:  385 loss:  0.008044405840337276
Iteration:  386 loss:  0.007260564249008894
Iteration:  387 loss:  0.007807706017047167
Iteration:  388 loss:  0.008063594810664654
Iteration:  389 loss:  0.008382942527532578
Iteration:  390 loss:  0.007294681388884783
Iteration:  391 loss:  0.006957044824957848
Iteration:  392 loss:  0.007379575166851282
Iteration:  393 loss:  0.006249243393540382
Iteration:  394 loss:  0.0052285403944551945
Iteration:  395 loss:  0.004912988748401403
Iteration:  396 loss:  0.005730575416237116
Iteration:  397 loss:  0.006592223886400461
Iteration:  398 loss:  0.00692874938249588
Iteration:  399 loss:  0.005397865

Iteration:  564 loss:  0.00664640823379159
Iteration:  565 loss:  0.006703627295792103
Iteration:  566 loss:  0.007076142355799675
Iteration:  567 loss:  0.006108908914029598
Iteration:  568 loss:  0.006003437098115683
Iteration:  569 loss:  0.005308340769261122
Iteration:  570 loss:  0.005717694293707609
Iteration:  571 loss:  0.005448104813694954
Iteration:  572 loss:  0.0060028922744095325
Iteration:  573 loss:  0.004908631555736065
Iteration:  574 loss:  0.004886074457317591
Iteration:  575 loss:  0.005675213411450386
Iteration:  576 loss:  0.006057743448764086
Iteration:  577 loss:  0.0074386452324688435
Iteration:  578 loss:  0.007510538212954998
Iteration:  579 loss:  0.006653910037130117
Iteration:  580 loss:  0.006333376746624708
Iteration:  581 loss:  0.005418118089437485
Iteration:  582 loss:  0.00590846873819828
Iteration:  583 loss:  0.005616751965135336
Iteration:  584 loss:  0.0062012625858187675
Iteration:  585 loss:  0.005434509366750717
Iteration:  586 loss:  0.008248

...

Iteration:  4043 loss:  0.004542908631265163
Iteration:  4044 loss:  0.004215562716126442
Iteration:  4045 loss:  0.004341085907071829
Iteration:  4046 loss:  0.0045316945761442184
Iteration:  4047 loss:  0.004174832720309496
Iteration:  4048 loss:  0.004677671007812023
Iteration:  4049 loss:  0.004575848579406738
Iteration:  4050 loss:  0.0042564840987324715
Iteration:  4051 loss:  0.00505813118070364
Iteration:  4052 loss:  0.004851360339671373
Iteration:  4053 loss:  0.00493867602199316
Iteration:  4054 loss:  0.003883349010720849
Iteration:  4055 loss:  0.003930279985070229
Iteration:  4056 loss:  0.00495968759059906
Iteration:  4057 loss:  0.004057449288666248
Iteration:  4058 loss:  0.004253176506608725
Iteration:  4059 loss:  0.0039292918518185616
Iteration:  4060 loss:  0.0055933608673512936
Iteration:  4061 loss:  0.0050397105515003204
Iteration:  4062 loss:  0.004827883560210466
Iteration:  4063 loss:  0.004771143198013306
Iteration:  4064 loss:  0.0054852343164384365
Iterati

Iteration:  4225 loss:  0.006055187899619341
Iteration:  4226 loss:  0.006107772700488567
Iteration:  4227 loss:  0.005319602321833372
Iteration:  4228 loss:  0.0057730949483811855
Iteration:  4229 loss:  0.004675562027841806
Iteration:  4230 loss:  0.005017663352191448
Iteration:  4231 loss:  0.005189391318708658
Iteration:  4232 loss:  0.005154681857675314
Iteration:  4233 loss:  0.005706885829567909
Iteration:  4234 loss:  0.004632595926523209
Iteration:  4235 loss:  0.005533288698643446
Iteration:  4236 loss:  0.004718855954706669
Iteration:  4237 loss:  0.004959032405167818
Iteration:  4238 loss:  0.006189011037349701
Iteration:  4239 loss:  0.0054322038777172565
Iteration:  4240 loss:  0.004832196980714798
Iteration:  4241 loss:  0.005536296404898167
Iteration:  4242 loss:  0.004738365299999714
Iteration:  4243 loss:  0.005298015661537647
Iteration:  4244 loss:  0.004765352234244347
Iteration:  4245 loss:  0.003694751299917698
Iteration:  4246 loss:  0.004857095889747143
Iteratio

Iteration:  4407 loss:  0.0045592593960464
Iteration:  4408 loss:  0.005535597912967205
Iteration:  4409 loss:  0.005357277113944292
Iteration:  4410 loss:  0.006319302599877119
Iteration:  4411 loss:  0.005502045154571533
Iteration:  4412 loss:  0.005318544805049896
Iteration:  4413 loss:  0.006197442766278982
Iteration:  4414 loss:  0.005883012432605028
Iteration:  4415 loss:  0.00595033960416913
Iteration:  4416 loss:  0.005321341101080179
Iteration:  4417 loss:  0.006213814485818148
Iteration:  4418 loss:  0.0062857335433363914
Iteration:  4419 loss:  0.006679309066385031
Iteration:  4420 loss:  0.006270224694162607
Iteration:  4421 loss:  0.005077951587736607
Iteration:  4422 loss:  0.005152011290192604
Iteration:  4423 loss:  0.0048188683576881886
Iteration:  4424 loss:  0.005540888290852308
Iteration:  4425 loss:  0.005529033951461315
Iteration:  4426 loss:  0.005521912127733231
Iteration:  4427 loss:  0.005962807219475508
Iteration:  4428 loss:  0.0054314397275447845
Iteration:

Iteration:  4589 loss:  0.004821478854864836
Iteration:  4590 loss:  0.0045743971131742
Iteration:  4591 loss:  0.004201579373329878
Iteration:  4592 loss:  0.004815470427274704
Iteration:  4593 loss:  0.0043905433267354965
Iteration:  4594 loss:  0.004624311346560717
Iteration:  4595 loss:  0.004968058317899704
Iteration:  4596 loss:  0.004868600517511368
Iteration:  4597 loss:  0.0049314945936203
Iteration:  4598 loss:  0.004881641361862421
Iteration:  4599 loss:  0.005337994080036879
Iteration:  4600 loss:  0.004559411201626062
Iteration:  4601 loss:  0.004526399075984955
Iteration:  4602 loss:  0.004518681205809116
Iteration:  4603 loss:  0.0036819246597588062
Iteration:  4604 loss:  0.0047106449492275715
Iteration:  4605 loss:  0.0049642049707472324
Iteration:  4606 loss:  0.003970996476709843
Iteration:  4607 loss:  0.004364712163805962
Iteration:  4608 loss:  0.004090525675565004
Iteration:  4609 loss:  0.0045296321623027325
Iteration:  4610 loss:  0.0046492828987538815
Iteratio

Iteration:  4771 loss:  0.0050629135221242905
Iteration:  4772 loss:  0.004385283682495356
Iteration:  4773 loss:  0.004926000256091356
Iteration:  4774 loss:  0.0049475980922579765
Iteration:  4775 loss:  0.004754719324409962
Iteration:  4776 loss:  0.0045379516668617725
Iteration:  4777 loss:  0.005298808217048645
Iteration:  4778 loss:  0.006793747190386057
Iteration:  4779 loss:  0.00664155138656497
Iteration:  4780 loss:  0.006234888453036547
Iteration:  4781 loss:  0.007151870056986809
Iteration:  4782 loss:  0.006856296677142382
Iteration:  4783 loss:  0.00664988998323679
Iteration:  4784 loss:  0.0055462149903178215
Iteration:  4785 loss:  0.006011465564370155
Iteration:  4786 loss:  0.004709659609943628
Iteration:  4787 loss:  0.004668776877224445
Iteration:  4788 loss:  0.003706202609464526
Iteration:  4789 loss:  0.004637850448489189
Iteration:  4790 loss:  0.004675485659390688
Iteration:  4791 loss:  0.004149492830038071
Iteration:  4792 loss:  0.00435960479080677
Iteration